#### Take a look at model registry and decide which model should be promoted to staging

In [21]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [22]:
client.create_experiment(name='my-experiment-2')

'4'

In [23]:
client.search_experiments()

[<Experiment: artifact_location='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/4', creation_time=1686331524486, experiment_id='4', last_update_time=1686331524486, lifecycle_stage='active', name='my-experiment-2', tags={}>,
 <Experiment: artifact_location='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2', creation_time=1686172186899, experiment_id='2', last_update_time=1686172186899, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 5.6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)
runs

[<Run: data=<RunData: metrics={'rmse': 5.110087914942558}, params={'learning_rate': '0.1186097146922628',
  'max_depth': '10',
  'min_child_weight': '1.2670591294263422',
  'objective': 'reg:linear',
  'reg_alpha': '0.2726478524166285',
  'reg_lambda': '0.007451987452974171',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "53e9be922bb74db4b39cd469c3a882f8", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2023-06-08 '
                              '21:54:07.420493", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.16", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"1.7.5", "data": "model.xgb", "model_class": '
                   

In [32]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 53e9be922bb74db4b39cd469c3a882f8, rmse: 5.1101
run id: 426977b423c448afa04ccd63f1f850ac, rmse: 5.1101
run id: dc3c3bc3ca7749ad99e9d03db90eae01, rmse: 5.1101
run id: 6952790856ce48aea99502dd6bbee60f, rmse: 5.1146
run id: 6da748baa6324412ab637f57fab15772, rmse: 5.1201


### promote models to model registry


In [34]:
import mlflow

mlflow.set_registry_uri(MLFLOW_TRACKING_URI)

In [35]:
run_id = "53e9be922bb74db4b39cd469c3a882f8"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/06/09 19:36:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1686332186702, current_stage='None', description=None, last_updated_timestamp=1686332186702, name='nyc-taxi-regressor', run_id='53e9be922bb74db4b39cd469c3a882f8', run_link=None, source='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4b39cd469c3a882f8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [38]:
# How to transition to staging

client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1686328401049, description='This is NYC taxi predictor for trip duration', last_updated_timestamp=1686332186702, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1686328673883, current_stage='Staging', description='', last_updated_timestamp=1686328874540, name='nyc-taxi-regressor', run_id='53e9be922bb74db4b39cd469c3a882f8', run_link='', source='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4b39cd469c3a882f8/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1686332186702, current_stage='None', description=None, last_updated_timestamp=1686332186702, name='nyc-taxi-regressor', run_id='53e9be922bb74db4b39cd469c3a882f8', run_link=None, source='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4

In [41]:
model_name = 'nyc-taxi-regressor'
lastest_versions = client.get_latest_versions(name = model_name)

for version in lastest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [43]:
model_version = 3
new_stage = "Staging"

In [50]:

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1686332186702, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-06-09', last_updated_timestamp=1686333225472, name='nyc-taxi-regressor', run_id='53e9be922bb74db4b39cd469c3a882f8', run_link=None, source='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4b39cd469c3a882f8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [49]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1686332186702, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-06-09', last_updated_timestamp=1686333193490, name='nyc-taxi-regressor', run_id='53e9be922bb74db4b39cd469c3a882f8', run_link=None, source='/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4b39cd469c3a882f8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Promote the model to production

In [51]:
from sklearn.metrics import mean_squared_error
import pandas as pd

In [60]:
def read_dataframe(filename):
    """read data
    """
    df = pd.read_parquet(filename)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    """ data wrangling
    """
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    """ test model with mlflow
    """
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}
    

In [61]:
df = read_dataframe('../data/green_tripdata_2023-03.parquet')

In [62]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/var/folders/y1/h3rdxlrj2qvdwwjh6r3b721m0000gp/T/ipykernel_97793/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/preprocessor'

In [63]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)
    

In [64]:
X_test = preprocess(df, dv)

In [65]:
target = 'duration'
y_test = df[target].values

In [67]:
%time
test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.82 µs


OSError: No such file or directory: '/Users/tdafonseca/Desktop/Github/learning/deploy_machine_learning_model/02-experiment-tracking/mlruns/2/53e9be922bb74db4b39cd469c3a882f8/artifacts/model/.'

'nyc-taxi-regressor'